# The Pacific Northwest

In [ ]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import librosa
import os

from IPython.display import Audio

IMAGE_PATH = 'images/'
SOUND_PATH = 'sounds/'

### 1.1 Correlation in Signals

In [ ]:
flicker, flicker_sr = librosa.load(SOUND_PATH + 'flicker.mp3')

display(Audio(flicker, rate=flicker_sr))

https://www.youtube.com/watch?v=YQ2wIOcO7aE&ab_channel=AppleseedOutdoors

In [ ]:
def calculate_sound_autocorrelation(y, sr, max_lag):
    # Calculate autocorrelation using numpy
    autocorr = np.correlate(y, y, mode='full')
    autocorr = autocorr[autocorr.size // 2:]  # Keep only the second half
    
    # Normalize the autocorrelation
    autocorr /= autocorr.max()

    # Consider only lags up to max_lag
    lags = np.arange(0, max_lag)
    autocorr = autocorr[:max_lag]

    return lags, autocorr

In [ ]:
lags, autocorr = calculate_sound_autocorrelation(flicker, flicker_sr, flicker_sr)

plt.figure(figsize=(10, 4))
plt.plot(lags, autocorr)
plt.xlabel('Lag (samples)')
plt.ylabel('Autocorrelation')
plt.title('Autocorrelation of Flicker Bird Call')
plt.show()

#### Measuring periodicity numerically

In [ ]:
from scipy.signal import find_peaks

peak_samples, peak_correlations = find_peaks(autocorr, height=0.03, distance=1000)

In [ ]:
# plot autocorrelation with samples marked
plt.figure(figsize=(10, 4))
plt.plot(lags, autocorr)

# mark the samples with red bars
plt.vlines(peak_samples, -1, 1, color='r')

In [ ]:
# average difference between peaks
avg_peak_diff = np.diff(peak_samples).mean()
flicker_shifted = np.roll(flicker, int(avg_peak_diff))

# plot the original signal
plt.figure(figsize=(10, 4))
plt.plot(flicker)
plt.plot(flicker_shifted, color='r')
plt.xlabel('Sample')
plt.ylabel('Amplitude')
plt.title('Flicker Bird Call')
plt.show()

In [ ]:
sample_window = (4500, 5500)
flicker_sample = flicker[sample_window[0]:sample_window[1]]

# plot the sample
plt.figure(figsize=(10, 4))
plt.plot(flicker)
plt.vlines(sample_window, -1, 1, color='r')
plt.xlabel('Sample')
plt.ylabel('Amplitude')
plt.title('Flicker Bird Call Sample')
plt.show()

In [ ]:
# find the flicker_sample in the flciker signal with cross-correlation
cross_corr = np.correlate(flicker, flicker_sample, mode='full')
#cross_corr = cross_corr[cross_corr.size // 2:]  # Keep only the second half

# normalize the cross-correlation
cross_corr /= cross_corr.max()

# plot the cross-correlation
plt.figure(figsize=(10, 4))
plt.plot(cross_corr)
plt.xlabel('Lag (samples)')
plt.ylabel('Cross-correlation')
plt.title('Cross-correlation of Flicker Bird Call Sample')
plt.show()


In [ ]:
sample_position = cross_corr.argmax()

# plot the flicker signal and mark the sample_position with a vline
plt.figure(figsize=(10, 4))
plt.plot(flicker)
plt.vlines(sample_position, -1, 1, color='r')
plt.xlabel('Sample')
plt.ylabel('Amplitude')
plt.title('Flicker Bird Call')
plt.show()

### 1.2 Segmentation, morphological operations, object attributes in images

Segmentation Techniques:
- Threshold-based segmentation
- Edge-based image segmentation
- Region-based image segmentation
- Clustering-based image segmentation
- Artificial neural network-based segmentation

In [ ]:
# load new image
bear_family = cv2.imread(IMAGE_PATH + 'brown_bear_family.png')

plt.imshow(cv2.cvtColor(bear_family, cv2.COLOR_BGR2RGB))

Source: https://my-photo-blog.com/brown-bear-family

In [ ]:
from skimage import io, color, exposure

# Load the image
image_path = IMAGE_PATH + 'brown_bear_family.png'
image = io.imread(image_path)

# Check if the image has an alpha channel (4th channel)
if image.shape[-1] == 4:
    # Discard the alpha channel
    image = image[..., :3]

# Convert image to grayscale
gray_image = color.rgb2gray(image)

# Increase contrast using histogram equalization
equalized_image = exposure.equalize_adapthist(gray_image, clip_limit=0.03)

# Display the original and the contrast-enhanced images
fig, ax = plt.subplots(1, 2, figsize=(12, 5))

# Original image
ax[0].imshow(gray_image, cmap='gray')
ax[0].set_title('Original Grayscale Image')
ax[0].axis('off')

# Contrast-enhanced image
ax[1].imshow(equalized_image, cmap='gray')
ax[1].set_title('Contrast Enhanced Image')
ax[1].axis('off')

plt.tight_layout()
plt.show()


In [ ]:

test_filter = np.array([[1,0,0],
                        [1,1,0],
                        [1,0,0]])

# apply the filter to the image
filtered_image = cv2.filter2D(equalized_image, -1, test_filter)

# plot both images
fig, ax = plt.subplots(1, 2, figsize=(12, 5))

# original image
ax[0].imshow(equalized_image, cmap='gray')
ax[0].set_title('Original Image')
ax[0].axis('off')

In [ ]:
# binarize the image of the bear fmaily
bear_family_gray = cv2.cvtColor(bear_family, cv2.COLOR_BGR2GRAY)
bear_family_bin = bear_family_gray > 100

plt.imshow(bear_family_bin, cmap='gray')


In [ ]:
# apply erosion and dilation
kernel = np.ones((9, 9), np.uint8)
bear_family_bin_eroded = cv2.erode(bear_family_bin.astype(np.uint8), kernel, iterations=4)
bear_family_bin_eroded_dilated = cv2.dilate(bear_family_bin_eroded, kernel, iterations=4)

# plot the binarized image
plt.imshow(bear_family_bin_eroded_dilated, cmap='gray')

### 2.1 Keypoint Matching

In [ ]:
def plot_images_from_folder(folder_path):
    image_extensions = ['.jpg', '.jpeg', '.png', '.gif', '.bmp']
    
    images = []
    for file in os.listdir(folder_path):
        if any(file.endswith(ext) for ext in image_extensions):
            img_path = os.path.join(folder_path, file)
            images.append(cv2.imread(img_path))

    # Plot images
    plt.figure(figsize=(10, 10))
    for i, img in enumerate(images, 1):
        plt.subplot(len(images), 1, i)
        plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
        plt.axis('off')
    plt.tight_layout()
    plt.show()

In [ ]:
plot_images_from_folder(IMAGE_PATH + 'keypoint_matching')